<a href="https://colab.research.google.com/github/EmKelsall/ADS2002---Catheter-Placement-Project/blob/main/SVM_2D_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report
from PIL import Image

In [3]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ADS2002-Project-CatheterPlacement



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1sDmXQTrvwQhCF8mWDh9phnwM9axaYjjb/ADS2002-Project-CatheterPlacement


In [3]:
# Define the image dimensions for 2D data
image_width = 600
image_height = 600
num_channels = 3  # Assuming RGB images

In [4]:
filenames = os.listdir("Data/train")
train_imgs = np.empty((0, image_width, image_height, num_channels))
FileIDs = []


In [10]:
# Load and preprocess your 2D images
# filenames = os.listdir("Data/train")
# train_imgs = np.empty((0, 600, 600, 3))  # Assuming 3 channels (RGB)
# FileIDs = []
# lst = []

old:

In [15]:
count = 0
for file in filenames:
    if count == 500:
        break

    # Making sure the images are the same pixel size
    img = Image.open(f"/content/drive/MyDrive/ADS2002-Project-CatheterPlacement/Data/train/{file}")
    img = img.resize((600, 600))

    # Convert the image to RGB if it's not already
    if img.mode != 'RGB':
        img = img.convert('RGB')

    img = np.array(img)

    # Save filename for later
    FileIDs.append(file)

    # Adding to train_imgs array
    train_imgs = np.concatenate((train_imgs, img.reshape(1, 600, 600, 3)), axis=0)

    count += 1

new:

In [5]:
count = 0
for file in filenames:
    if count == 500:
        break

    # Load and resize the image
    img = Image.open(f"/content/drive/MyDrive/ADS2002-Project-CatheterPlacement/Data/train/{file}")
    img = img.resize((image_width, image_height))

    # Ensure the image has the correct number of channels (RGB)
    img = img.convert("RGB")

    # Convert the image to a NumPy array
    img = np.array(img)

    # Save filename for later
    FileIDs.append(file)

    # Adding to train_imgs array
    train_imgs = np.concatenate((train_imgs, img.reshape(1, image_width, image_height, num_channels)), axis=0)

    count += 1


In [6]:
# Load only CVC labels
train_data = pd.read_csv("/content/drive/MyDrive/ADS2002-Project-CatheterPlacement/Data/train.csv")
CVC_labels = train_data[["StudyInstanceUID", "CVC - Abnormal", "CVC - Borderline", "CVC - Normal"]]
CVC_labels.set_index('StudyInstanceUID', inplace=True)

# Match labels with loaded images based on filenames
UID = [file.replace(".jpg", "") for file in FileIDs]
train_labels_df = CVC_labels.loc[CVC_labels.index.isin(UID)]
train_labels = train_labels_df.values


In [6]:
# # Load labels or any other required data
# train_data = pd.read_csv("/content/drive/MyDrive/ADS2002-Project-CatheterPlacement/Data/train.csv")

# # Assuming your labels are encoded as 0, 1, 2
# UID = [file.replace(".jpg", "") for file in FileIDs]
# train_labels_df = train_data.loc[train_data['StudyInstanceUID'].isin(UID)][["StudyInstanceUID", "CVC - Abnormal", "CVC - Borderline", "CVC - Normal"]]
# train_labels_df.set_index('StudyInstanceUID', inplace=True)
# train_labels = train_labels_df.values

In [7]:
# Assuming you have labels encoded as 0, 1, 2
# Flatten the images in train_imgs
train_imgs_flattened = train_imgs.reshape(train_imgs.shape[0], -1)


In [8]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(train_imgs_flattened, train_labels, test_size=0.2, random_state=42)


In [11]:
# Convert one-hot encoded labels to class indices
Y_train_indices = np.argmax(Y_train, axis=1)
Y_test_indices = np.argmax(Y_test, axis=1)


In [1]:
# Initialize an SVM classifier
from sklearn import svm
svm_classifier = svm.SVC(kernel='linear')  # You can choose the kernel type you prefer

# Train the SVM model with your data and labels
svm_classifier.fit(X_train, Y_train_indices)  # Use Y_train_indices instead of Y_train

# Make predictions on the test set
y_pred_indices = svm_classifier.predict(X_test)

# Evaluate the model using accuracy and classification report
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(Y_test_indices, y_pred_indices)  # Use Y_test_indices for evaluation
classification_rep = classification_report(Y_test_indices, y_pred_indices)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)


NameError: ignored